<a href="https://colab.research.google.com/github/giovanecs/mvp-water-quality/blob/main/MVP_WATER_QUALITY_ESSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Este notebook tem por objetivo contruir um modelo machine learning para identificar se determinada amostra de água é ou não potavél.

## Importação de bibliotecas



In [2]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from pickle import dump
from pickle import load

## Carga do Dataset

In [3]:
# Informa a URL de importação do dataset
url = "https://raw.githubusercontent.com/giovanecs/datasets/main/water_potability.csv"

# Lê o arquivo
dataset = pd.read_csv(url, delimiter=',')


# Mostra as primeiras linhas do dataset
dataset.head(10)

,ph,hardness,solids,chloramines,sulfate,conductivity,organic_carbon,trihalomethanes,turbidity,potability
0,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
1,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
2,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,0
3,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989,0
4,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,0
5,11.180284,227.231469,25484.508491,9.077200,404.041635,563.885481,17.927806,71.976601,4.370562,0
6,7.360640,165.520797,32452.614409,7.550701,326.624353,425.383419,15.586810,78.740016,3.662292,0
7,7.119824,156.704993,18730.813653,3.606036,282.344050,347.715027,15.929536,79.500778,3.445756,0
8,6.347272,186.732881,41065.234765,9.629596,364.487687,516.743282,11.539781,75.071617,4.376348,0
9,9.181560,273.813807,24041.326280,6.904990,398.350517,477.974642,13.387341,71.457362,4.503661,0


## Separação em conjunto de treino e conjunto de teste com holdout

In [4]:
# Separando os dados em features (X) e rótulos (y)
X = dataset.drop('potability', axis=1)
y = dataset['potability']

# Definindo a semente global para garantir reprodutibilidade
seed = 7

# Separando em conjuntos de treino e teste usando holdout
test_size = 0.2  # proporção para o conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=test_size,
    random_state=seed,
    stratify=y  # garantindo que a proporção de classes seja mantida nos conjuntos de treino e teste
)

# Parâmetros e partições da validação cruzada
scoring = 'accuracy'
num_particoes = 10
kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=seed) # validação cruzada com estratificação

# Exibindo informações sobre os conjuntos
print(f"Tamanho do conjunto de treino: {len(X_train)} amostras")
print(f"Tamanho do conjunto de teste: {len(X_test)} amostras")

df = pd.DataFrame(X_test)
df.head()

Tamanho do conjunto de treino: 1608 amostras
Tamanho do conjunto de teste: 403 amostras


,ph,hardness,solids,chloramines,sulfate,conductivity,organic_carbon,trihalomethanes,turbidity
247,5.040332,232.234479,25653.690231,5.929308,328.329580,529.052535,13.539410,38.346738,3.603326
949,6.642756,168.896422,36505.524258,6.062561,298.552480,474.057490,14.299242,71.706891,4.299209
1662,6.991685,152.976217,20389.593816,2.648390,331.252916,460.146174,15.485378,69.670038,3.724824
299,4.370477,188.654037,29542.338165,8.117114,301.022182,456.308454,16.346838,46.696941,4.649382
722,5.728824,197.191839,12932.590386,5.684492,280.932064,445.069636,14.176501,68.124809,4.412174


## Modelagem e Inferência

### Criação e avaliação de modelos: linha base

In [5]:
np.random.seed(7) # definindo uma semente global

# Lista que armazenará os modelos
models = []

# Criando os modelos e adicionando-os na lista de modelos
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# Listas para armazenar os resultados
results = []
names = []

# Avaliação dos modelos
for name, model in models:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

KNN: 0.558459 (0.035256)
CART: 0.587721 (0.035681)
NB: 0.619422 (0.033020)
SVM: 0.597019 (0.001491)


### Criação e avaliação de modelos: dados padronizados e normalizados

In [6]:
np.random.seed(7) # definindo uma semente global para este bloco

# Listas para armazenar os armazenar os pipelines e os resultados para todas as visões do dataset
pipelines = []
results = []
names = []


# Criando os elementos do pipeline

# Algoritmos que serão utilizados
knn = ('KNN', KNeighborsClassifier())
cart = ('CART', DecisionTreeClassifier())
naive_bayes = ('NB', GaussianNB())
svm = ('SVM', SVC())

# Transformações que serão utilizadas
standard_scaler = ('StandardScaler', StandardScaler())
min_max_scaler = ('MinMaxScaler', MinMaxScaler())


# Montando os pipelines

# Dataset original
pipelines.append(('KNN-orig', Pipeline([knn])))
pipelines.append(('CART-orig', Pipeline([cart])))
pipelines.append(('NB-orig', Pipeline([naive_bayes])))
pipelines.append(('SVM-orig', Pipeline([svm])))

# Dataset Padronizado
pipelines.append(('KNN-padr', Pipeline([standard_scaler, knn])))
pipelines.append(('CART-padr', Pipeline([standard_scaler, cart])))
pipelines.append(('NB-padr', Pipeline([standard_scaler, naive_bayes])))
pipelines.append(('SVM-padr', Pipeline([standard_scaler, svm])))

# Dataset Normalizado
pipelines.append(('KNN-norm', Pipeline([min_max_scaler, knn])))
pipelines.append(('CART-norm', Pipeline([min_max_scaler, cart])))
pipelines.append(('NB-norm', Pipeline([min_max_scaler, naive_bayes])))
pipelines.append(('SVM-norm', Pipeline([min_max_scaler, svm])))

# Executando os pipelines
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %.2f (%.2f)" % (name, cv_results.mean(), cv_results.std()) # formatando para 2 casas decimais
    print(msg)

KNN-orig: 0.56 (0.04)
CART-orig: 0.59 (0.04)
NB-orig: 0.62 (0.03)
SVM-orig: 0.60 (0.00)
KNN-padr: 0.64 (0.03)
CART-padr: 0.58 (0.05)
NB-padr: 0.62 (0.04)
SVM-padr: 0.68 (0.02)
KNN-norm: 0.61 (0.02)
CART-norm: 0.59 (0.03)
NB-norm: 0.62 (0.04)
SVM-norm: 0.68 (0.02)


### Otimização dos hiperparâmetros

In [7]:
# Definindo a semente global para garantir reprodutibilidade
np.random.seed(7)

### otimização do modelo KNN

In [8]:
# Criando o modelo KNN
knn = KNeighborsClassifier()

# Criando o pipeline com o escalonador e o modelo
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', knn)
])

# Definindo a grade de hiperparâmetros a serem testados
param_grid = {
    'knn__n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21],
    'knn__metric': ["euclidean", "manhattan", "minkowski"],
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1, 2]
}

# Criando o objeto StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='accuracy', cv=kfold)

# Realizando a busca em grade
grid_result = grid_search.fit(X_train, y_train)

# Exibindo os resultados
print("Melhores Parâmetros: ", grid_result.best_params_)
print("Melhor Acurácia Média: ", grid_result.best_score_)

Melhores Parâmetros:  {'knn__metric': 'euclidean', 'knn__n_neighbors': 17, 'knn__p': 1, 'knn__weights': 'distance'}
Melhor Acurácia Média:  0.6604697204968943


### otimização do algoritmo Decision Tree


In [9]:
# Criando o modelo Decision Tree
decision_tree = DecisionTreeClassifier()

# Criando o pipeline com o escalonador e o modelo Decision Tree
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('decision_tree', decision_tree)
])

# Definindo a grade de hiperparâmetros a serem testados
param_grid = {
    'decision_tree__criterion': ['gini', 'entropy'],
    'decision_tree__splitter': ['best', 'random'],
    'decision_tree__max_depth': [None, 5, 10, 15, 20],
    'decision_tree__min_samples_split': [2, 5, 10],
    'decision_tree__min_samples_leaf': [1, 2, 4],
    'decision_tree__max_features': ['sqrt', 'log2']
}

# Criando o objeto StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='accuracy', cv=kfold)

# Realizando a busca em grade
grid_result = grid_search.fit(X_train, y_train)

# Exibindo os resultados
print("Melhores Parâmetros: ", grid_result.best_params_)
print("Melhor Acurácia Média: ", grid_result.best_score_)


Melhores Parâmetros:  {'decision_tree__criterion': 'entropy', 'decision_tree__max_depth': 5, 'decision_tree__max_features': 'sqrt', 'decision_tree__min_samples_leaf': 4, 'decision_tree__min_samples_split': 2, 'decision_tree__splitter': 'best'}
Melhor Acurácia Média:  0.6399495341614908


### otimização do algoritmo Gaussian Naive Bayes

In [10]:
# Criando o modelo Gaussian Naive Bayes
naive_bayes = GaussianNB()

# Criando o pipeline com o escalonador e o modelo Gaussian Naive Bayes
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('naive_bayes', naive_bayes)
])

# Definindo a grade de hiperparâmetros a serem testados
param_grid = {
    'naive_bayes__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

# Criando o objeto StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='accuracy', cv=kfold)

# Realizando a busca em grade
grid_result = grid_search.fit(X_train, y_train)

# Exibindo os resultados
print("Melhores Parâmetros: ", grid_result.best_params_)
print("Melhor Acurácia Média: ", grid_result.best_score_)

Melhores Parâmetros:  {'naive_bayes__var_smoothing': 1e-09}
Melhor Acurácia Média:  0.6206715838509316


### otimização do algoritmo SVM

In [12]:
# Criando o modelo Support Vector Machine (SVM)
svm_classifier = SVC()

# Criando o pipeline com o escalonador e o modelo SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', svm_classifier)
])

# Definindo a grade de hiperparâmetros a serem testados para o SVM
param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf', 'poly'],
    'svm__gamma': ['scale', 'auto']
}

# Criando o objeto StratifiedKFold
kfold = StratifiedKFold(n_splits=10, shuffle=True)

# Criando o objeto GridSearchCV
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, scoring='accuracy', cv=kfold)

# Realizando a busca em grade
grid_result = grid_search.fit(X_train, y_train)

# Exibindo os resultados
print("Melhores Parâmetros: ", grid_result.best_params_)
print("Melhor Acurácia Média: ", grid_result.best_score_)


Melhores Parâmetros:  {'svm__C': 1, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Melhor Acurácia Média:  0.6809627329192546


## Finalização do Modelo

In [13]:
# Padronização dos dados
scaler = StandardScaler().fit(X_train)
rescaledX_train = scaler.transform(X_train)
rescaledX_test = scaler.transform(X_test)

# Preparação do modelo SVC com os parâmetros específicos
model = SVC(C=1, kernel='rbf', gamma='scale')

# Treinamento do modelo
model.fit(rescaledX_train, y_train)

# Estimativa da acurácia no conjunto de teste
predictions = model.predict(rescaledX_test)
accuracy = accuracy_score(y_test, predictions)
print("Acurácia no Conjunto de Teste: ", accuracy)

Acurácia no Conjunto de Teste:  0.7047146401985112


In [14]:
# Salva o modelo no disco
filename = 'model.pkl'
dump(model, open(filename, 'wb'))


## Simulando a aplicação do modelo em dados não vistos

In [36]:
# Novos dados - não sabemos a classe!
data = {
    'ph': [6.376408434486552, 8.0, 7.2],
    'hardness': [182.1210381950736, 210.5, 198.3],
    'solids': [33046.727605817854, 21500.2, 21950.8],
    'chloramines': [5.35153318891046, 8.2, 7.5],
    'sulfate': [362.35214343376356, 360.5, 355.2],
    'conductivity': [350.23632637500947, 365.2, 362.8],
    'organic_carbon': [7.5925157501404215, 19.8, 21.0],
    'trihalomethanes': [69.22567377981284, 101.2, 99.5],
    'turbidity': [5.404045772985421, 4.5, 3.8],
}

# Criando um DataFrame com os novos dados simulados
new_df = pd.DataFrame(data)

# Obtendo apenas as características presentes nos novos dados
new_features = new_df.columns

# Padronização dos novos dados simulados
new_rescaled = pd.DataFrame(scaler.transform(new_df), columns=new_features)

print(new_rescaled)

         ph  hardness    solids  chloramines   sulfate  conductivity  \
0 -0.446993 -0.405207  1.301214    -1.127374  0.696916     -0.948133   
1  0.585413  0.467075 -0.031552     0.675292  0.652084     -0.763150   
2  0.076711  0.092084  0.020459     0.232294  0.523795     -0.792820   

   organic_carbon  trihalomethanes  turbidity  
0       -2.049989         0.159606   1.834041  
1        1.684198         2.143669   0.675462  
2        2.051269         2.038181  -0.221622  


In [37]:
# Realizando previsões
predictions = model.predict(new_rescaled)
print(predictions)

[1 0 0]


 Após realizar testes e otimizações, o algoritmo SVM foi o escolhido para o modelo que servirá com base para a api, pois foi o que apresentou melhor acurácia.

